In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [9]:
def read_dataframe(file_path):
    df = pd.read_parquet(file_path)

    # Calculating the duration in seconds
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td : td.total_seconds() / 60)

    # Taking duration Morethan 1 minute and Lessthan 1 hour
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    # Categorical and Numerical Variables to use
    categorical = ['PULocationID', 'DOLocationID']

    # One hot encoing Categorical Variables using Dictionary Vectorizer
    df[categorical] = df[categorical].astype(str)
    
    return df

In [10]:
# Loading dataframes
df_train = read_dataframe('../data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('../data/yellow_tripdata_2023-02.parquet')

In [11]:
df_train.duration.std()

42.594351241920904

In [4]:
# List of columns to be used
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# Initializing DictVectorizer
dv = DictVectorizer()

# Collecting train dicts
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# Collecting Val dicts
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [5]:
# Collecting train and val y data
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [7]:
# Building Model
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

np.sqrt(mean_squared_error(y_val, y_pred))

7.649137702276996

In [33]:
# Saving the model
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

# Homework Solutions

https://www.notion.so/Home-Work-Solutions-976095ff8dbd4e03943601ab70629dff?pvs=4